In [1]:
from array import array
import reprlib
import math

In [2]:
class Vector:
    typecode = 'd'
    
    def __init__(self, componments):
        print('call __init__()')
        self._componments = array(self.typecode, componments)
        
    def __iter__(self):
        print('call __iter__()')
        return iter(self._componments)
    
    def __repr__(self):
        print('call __repr__()')
        componments = reprlib.repr(self._componments)
        componments = componments[componments.find('['):-1]
        return 'Vector({})'.format(componments)
    
    def __str__(self):
        print('call __str__()')
        return str(tuple(self))
    
    def __bytes__(self):
        print('call __bytes__()')
        return (bytes([ord(self.typecode)]) + bytes(self._componments))
    
    def __eq__(self, other):
        print('call __eq__()')
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        print('call __abs__()')
        return math.sqrt(sum(x**2 for x in self))
    
    def __bool__(self):
        print('call __bool__()')
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        print('call classmethod frombytes()')
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [3]:
v1 = Vector([3, 4])

call __init__()


In [4]:
v1

call __repr__()


Vector([3.0, 4.0])

In [5]:
v1._componments

array('d', [3.0, 4.0])

In [6]:
v2 = Vector(list(range(10)))

call __init__()


In [7]:
v2

call __repr__()


Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [8]:
repr(v2)

call __repr__()


'Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])'

## 可切片序列

In [9]:
class Vector:
    typecode = 'd'
    
    def __init__(self, componments):
        print('call __init__()')
        self._componments = array(self.typecode, componments)
        
    def __iter__(self):
        print('call __iter__()')
        return iter(self._componments)
    
    def __repr__(self):
        print('call __repr__()')
        componments = reprlib.repr(self._componments)
        componments = componments[componments.find('['):-1]
        return 'Vector({})'.format(componments)
    
    def __str__(self):
        print('call __str__()')
        return str(tuple(self))
    
    def __bytes__(self):
        print('call __bytes__()')
        return (bytes([ord(self.typecode)]) + bytes(self._componments))
    
    def __eq__(self, other):
        print('call __eq__()')
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        print('call __abs__()')
        return math.sqrt(sum(x**2 for x in self))
    
    def __bool__(self):
        print('call __bool__()')
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._componments[index]
    
    @classmethod
    def frombytes(cls, octets):
        print('call classmethod frombytes()')
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [10]:
v = Vector([3, 4])

call __init__()


In [11]:
v[0]

3.0

In [12]:
v[:]

array('d', [3.0, 4.0])

## slice

In [13]:
class MySeq:
    
    def __getitem__(self, index):
        return index

In [14]:
s = MySeq()

In [15]:
s[1]

1

**当我们输入切片的形式去取数据，\_\_getitem\_\_()方法会自动将其转换为slice对象，然后使用slice去取数据**

In [16]:
s[1:4]

slice(1, 4, None)

In [17]:
s[1:4:2]

slice(1, 4, 2)

In [18]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [19]:
s[1:4:2, 1:3]

(slice(1, 4, 2), slice(1, 3, None))

In [20]:
s[1:4:2, 2:8:3, 2]

(slice(1, 4, 2), slice(2, 8, 3), 2)

In [21]:
import numbers
import operator
import functools
import itertools

In [31]:
class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, componments):
        print('call __init__()')
        self._componments = array(self.typecode, componments)
        
    def __iter__(self):
        print('call __iter__()')
        return iter(self._componments)
    
    def __repr__(self):
        print('call __repr__()')
        componments = reprlib.repr(self._componments)
        componments = componments[componments.find('['):-1]
        return 'Vector({})'.format(componments)
    
    def __str__(self):
        print('call __str__()')
        return str(tuple(self))
    
    def __bytes__(self):
        print('call __bytes__()')
        return (bytes([ord(self.typecode)]) + bytes(self._componments))
    
    def __eq__(self, other):
        print('call __eq__()')
#         if len(self) != len(other):
#             return False
#         for a, b in zip(self, other):
#             if a != b:
#                 return False
#         return True
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __abs__(self):
        print('call __abs__()')
        return math.sqrt(sum(x**2 for x in self))
    
    def __bool__(self):
        print('call __bool__()')
        return bool(abs(self))
    
    def __len__(self):
        print('call __len__()')
        return len(self._componments)
    
    def __getitem__(self, index):
        print('call __getitem__()')
        cls = type(self) # 获取实例所属的类
        if isinstance(index, slice):  # 如果取的是一个序列的数据，那么仍然返回一个class
            return cls(self._componments[index])
        elif isinstance(index, numbers.Integral):
            return self._componments[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
    def __getattr__(self, name):
        print('call __getattr__()')
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._componments):
                return self._componments[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        print('call __setattr__()')
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
                
        super().__setattr__(name, value)
        
    def __hash__(self):  # 跟之前一样，自定义对象的hash值由其所有属性的异或运算得到
        print('call __hash__()')
        hashes = (hash(x) for x in self._componments)
        return functools.reduce(operator.xor, hashes, 0)
    
    def angle(self, n):
        r = math.sqrt(sum(x**2 for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
        
    @classmethod
    def frombytes(cls, octets):
        print('call classmethod frombytes()')
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [32]:
v7 = Vector(range(7))

call __init__()
call __setattr__()


In [33]:
v7[-1]

call __getitem__()


6.0

In [34]:
v7[1:4]

call __getitem__()
call __init__()
call __setattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __repr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()


Vector([1.0, 2.0, 3.0])

In [35]:
v7[-1:]

call __getitem__()
call __init__()
call __setattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __repr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()
call __getattr__()


Vector([6.0])

In [36]:
v7[1, 2]

call __getitem__()


TypeError: Vector indices must be integers

In [37]:
format(Vector([1, 1]), '.3fh')

call __init__()
call __setattr__()
call __abs__()
call __iter__()
call __len__()
call __getitem__()
call __init__()
call __setattr__()
call __iter__()
call __getitem__()
call __len__()
call __getitem__()


'<1.414, 0.785>'

In [38]:
format(Vector([2, 2, 2]), '.3fh')

call __init__()
call __setattr__()
call __abs__()
call __iter__()
call __len__()
call __getitem__()
call __init__()
call __setattr__()
call __iter__()
call __getitem__()
call __len__()
call __getitem__()
call __init__()
call __setattr__()
call __iter__()
call __getitem__()
call __len__()
call __getitem__()


'<3.464, 0.955, 0.785>'

In [40]:
v7.x, v7.y, v7.z

call __getattr__()
call __getattr__()
call __getattr__()


(0.0, 1.0, 2.0)

In [41]:
v7.x = 100

call __setattr__()


AttributeError: readonly attribute 'x'

In [95]:
class A:
    def __init__(self):
        self.a = [1, 2, 3]
        self.b = 5
    
    def print(self):
        print(self)

In [71]:
a = A()
a.print()

In [1]:
sum([])

0

In [2]:
sum()

TypeError: sum expected at least 1 arguments, got 0

In [59]:
class ToyClass:
    maybe_attrs = 'xyz'
    
    def __init__(self):
        self.ls = [1, 2, 3]
    
    
    def __getattr__(self, attr):
        cls = type(self)
        if len(attr) == 1:
            pos = cls.maybe_attrs.find(attr)
            if 0 <= pos < len(self.maybe_attrs):
                return self.ls[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, attr))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in self.maybe_attrs:
                error = "readonly attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

In [60]:
t = ToyClass()

In [61]:
t.x

1

In [62]:
t.x = 100

AttributeError: readonly attribute 'x'

In [ ]:
t.

In [64]:
t.X = 1000

In [65]:
t.ls

[1, 2, 3]